In [102]:
import os
import sys
import time
import pickle

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
import numpy as np

import allennlp
# from allennlp.common.testing import AllenNlpTestCase, ModelTestCase
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
# from Module import 

In [44]:
raw = pickle.load(open('rnn_input.txt', 'rb'))

In [45]:
print(raw[0:2])

[{'verbs': (('worked_with',), ['shortened', 'stylized_as', 'is', 'is_owned_by']), 'label': 'SUPPORTS'}, {'verbs': (('worked_with',), ['played', 'appearing_as', 'intended_as']), 'label': 'SUPPORTS'}]


In [46]:
x_train, y_train = [], []

In [47]:
for d in raw:
    x_train.append(d['verbs'])
    y_train.append(d['label'])

In [48]:
print(y_train[:100])

['SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT ENOUGH INFO', 'SUPPORTS', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'REFUTES', 'REFUTES', 'REFUTES', 'REFUTES', 'SUPPORTS', 'NOT ENOUGH INFO', 'SUPPORTS', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'NOT ENOUGH INFO', 'NOT ENOUGH INFO', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'NOT ENOUGH INFO', 'SUPPORTS', 'REFUTES', 'SUPPORTS', 'SUPPORTS', 'SUPPORTS', '

In [49]:
y_train = np.array(y_train)
print(y_train[:20])

['SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'REFUTES'
 'NOT ENOUGH INFO' 'SUPPORTS' 'SUPPORTS' 'NOT ENOUGH INFO'
 'NOT ENOUGH INFO' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS'
 'SUPPORTS' 'SUPPORTS' 'SUPPORTS' 'SUPPORTS']


In [50]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)
print(integer_encoded[:20])

[2 2 2 2 2 1 0 2 2 0 0 2 2 2 2 2 2 2 2 2]


In [52]:
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded[:20])

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:414: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [53]:
print(x_train[0])

(('worked_with',), ['shortened', 'stylized_as', 'is', 'is_owned_by'])


In [54]:
clf = SVC(gamma='auto')

# DatasetReader

In [114]:
import pickle
from nltk.tokenize import word_tokenize
from allennlp.data.tokenizers import Token
from allennlp.data.fields import TextField, LabelField
from typing import Iterator, List, Dict
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data import Vocabulary
from allennlp.data.dataset import Batch


class VerbDatasetReader(DatasetReader):

    def __init__(self,sentence_indexers:Dict[str,TokenIndexer]=None )-> None:
        super().__init__(lazy=False)
        self.sentence_indexers=sentence_indexers or {"sentence":SingleIdTokenIndexer()}

    def text_to_instance(self, claim: List[List], evidence: List[List], labels:str = None) -> Instance:
        evidence_tokenized, claim_tokenized = [], []
        for c in claim:
            for word in word_tokenize(c):
                claim_tokenized.append(Token(word))
        for e in evidence:
            for word in word_tokenize(e):
                evidence_tokenized.append(Token(word))
        claim_textfiled = TextField(claim_tokenized, self.sentence_indexers)
        evidence_textfiled = TextField(evidence_tokenized, self.sentence_indexers)
        fields={'claim':claim_textfiled, 'evidence':evidence_textfiled,'label': LabelField(labels)}
        return Instance(fields)

    def _read(self, file_path: str)->Iterator[Instance]:
        mlinput_merge=pickle.load(open(file_path,'rb'))
        for entry in mlinput_merge:
            claim = [entry['claim']]
            evidence = []
            for sentence in entry['evidence']:
                sentence = ' '.join(sentence)
                evidence.append(sentence)
            yield self.text_to_instance(claim, evidence, entry['label'])



reader=VerbDatasetReader()

instances = reader.read('mlinput_merge.txt')






145449it [02:09, 1125.97it/s]


In [115]:
vars(instances[0].fields['evidence'])

{'tokens': [The,
  Fox,
  Broadcasting,
  Company,
  -LRB-,
  often,
  shortened,
  to,
  Fox,
  and,
  stylized,
  as,
  FOX,
  -RRB-,
  is,
  an,
  American,
  English,
  language,
  commercial,
  broadcast,
  television,
  network,
  that,
  is,
  owned,
  by,
  the,
  Fox,
  Entertainment,
  Group,
  subsidiary,
  of,
  21st,
  Century,
  Fox,
  .,
  He,
  then,
  played,
  Detective,
  John,
  Amsterdam,
  in,
  the,
  short-lived,
  Fox,
  television,
  series,
  New,
  Amsterdam,
  -LRB-,
  2008,
  -RRB-,
  ,,
  as,
  well,
  as,
  appearing,
  as,
  Frank,
  Pike,
  in,
  the,
  2009,
  Fox,
  television,
  film,
  Virtuality,
  ,,
  originally,
  intended,
  as,
  a,
  pilot,
  .],
 '_token_indexers': {'sentence': <allennlp.data.token_indexers.single_id_token_indexer.SingleIdTokenIndexer at 0x17ac22c50>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

In [116]:
vocab = Vocabulary.from_instances(instances)

100%|██████████| 145449/145449 [00:06<00:00, 22042.80it/s]


# Embedding

In [117]:
test_size = 100

In [ ]:
embedding
token_embedding = Embedding(num_embeddings=config.max_vocab_size + 2,
                            embedding_dim=100, padding_index=0)
# the embedder maps the input tokens to the appropriate embedding matrix
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": token_embedding})

In [118]:
# Basic embeddings
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

claim_embedding = Embedding(num_embeddings=test_size, embedding_dim=100)
# evidence_embedding = Embedding(num_embeddings=vocab.get_vocab_size("evidence"), embedding_dim=100, padding_index=0)
evidence_embedding = Embedding(num_embeddings=test_size, embedding_dim=100, padding_index=0)

# the embedder maps the input tokens to the appropriate embedding matrix
word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": token_embedding})

AttributeError: module 'torch' has no attribute '_six'

# Batch

In [ ]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("claim", "num_tokens"), ("evidence", "num_tokens")],
                         )
iterator.index_with(vocab)

In [ ]:
batch = next(iter(iterator(training_set)))

# Model

In [108]:
class AcademicPaperClassifier(Model):
    def __init__(self,
                 vocab: Vocabulary,
                 text_field_embedder: TextFieldEmbedder,
                 title_encoder: Seq2VecEncoder,
                 abstract_encoder: Seq2VecEncoder,
                 classifier_feedforward: FeedForward,
                 initializer: InitializerApplicator = InitializerApplicator(),
                 regularizer: Optional[RegularizerApplicator] = None) -> None:
        super(AcademicPaperClassifier, self).__init__(vocab, regularizer)

        self.text_field_embedder = text_field_embedder
        self.num_classes = self.vocab.get_vocab_size("labels")
        self.title_encoder = title_encoder
        self.abstract_encoder = abstract_encoder
        self.classifier_feedforward = classifier_feedforward
        self.metrics = {
                "accuracy": CategoricalAccuracy(),
                "accuracy3": CategoricalAccuracy(top_k=3)
        }
        self.loss = torch.nn.CrossEntropyLoss()
        initializer(self)
        
    def forward(self, title: Dict[str, torch.LongTensor], abstract: Dict[str, torch.LongTensor],
                label: torch.LongTensor = None) -> Dict[str, torch.Tensor]:
        embedded_title = self.text_field_embedder(title)
        title_mask = util.get_text_field_mask(title)
        encoded_title = self.title_encoder(embedded_title, title_mask)

        embedded_abstract = self.text_field_embedder(abstract)
        abstract_mask = util.get_text_field_mask(abstract)
        encoded_abstract = self.abstract_encoder(embedded_abstract, abstract_mask)

        logits = self.classifier_feedforward(torch.cat([encoded_title, encoded_abstract], dim=-1))
        class_probabilities = F.softmax(logits)

        output_dict = {"class_probabilities": class_probabilities}

        if label is not None:
            loss = self.loss(logits, label.squeeze(-1))
            for metric in self.metrics.values():
                metric(logits, label.squeeze(-1))
            output_dict["loss"] = loss

        return output_dict

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {metric_name: metric.get_metric(reset) for metric_name, metric in self.metrics.items()}
This method is how the model tells the training code which metrics it's computing. The second pieces of code is the decode method:

    def decode(self, output_dict: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        predictions = output_dict['class_probabilities'].cpu().data.numpy()
        argmax_indices = numpy.argmax(predictions, axis=-1)
        labels = [self.vocab.get_token_from_index(x, namespace="labels")
                  for x in argmax_indices]
        output_dict['label'] = labels
        return output_dict

NameError: name 'Model' is not defined

In [61]:
print(train_dataset[0])

Instance with fields:
 	 evidece: TextField of length 85 with text: 
 		[Nikolaj, Coster-Waldau, worked, with, the, Fox, Broadcasting, Company, ., The, Fox, Broadcasting,
		Company, -LRB-, often, shortened, to, Fox, and, stylized, as, FOX, -RRB-, is, an, American, English,
		language, commercial, broadcast, television, network, that, is, owned, by, the, Fox, Entertainment,
		Group, subsidiary, of, 21st, Century, Fox, ., He, then, played, Detective, John, Amsterdam, in, the,
		short-lived, Fox, television, series, New, Amsterdam, -LRB-, 2008, -RRB-, ,, as, well, as,
		appearing, as, Frank, Pike, in, the, 2009, Fox, television, film, Virtuality, ,, originally,
		intended, as, a, pilot, .]
 		and TokenIndexers : {'sentence': 'SingleIdTokenIndexer'} 
 	 label: LabelField with label: SUPPORTS in namespace: 'labels'.' 



In [70]:
with open('mlinput_merge.txt', 'rb') as m:
    a = pickle.load(m)
    print(a[0])



{'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.', 'label': 'SUPPORTS', 'evidence': [['The', 'Fox', 'Broadcasting', 'Company', '-LRB-', 'often', 'shortened', 'to', 'Fox', 'and', 'stylized', 'as', 'FOX', '-RRB-', 'is', 'an', 'American', 'English', 'language', 'commercial', 'broadcast', 'television', 'network', 'that', 'is', 'owned', 'by', 'the', 'Fox', 'Entertainment', 'Group', 'subsidiary', 'of', '21st', 'Century', 'Fox', '.\n'], ['He', 'then', 'played', 'Detective', 'John', 'Amsterdam', 'in', 'the', 'short-lived', 'Fox', 'television', 'series', 'New', 'Amsterdam', '-LRB-', '2008', '-RRB-', ',', 'as', 'well', 'as', 'appearing', 'as', 'Frank', 'Pike', 'in', 'the', '2009', 'Fox', 'television', 'film', 'Virtuality', ',', 'originally', 'intended', 'as', 'a', 'pilot', '.\n']]}


In [ ]:
class TestSemanticScholarDatasetReader(AllenNlpTestCase):
    def test_read_from_file(self):
        reader = SemanticScholarDatasetReader()
        dataset = reader.read('tests/fixtures/s2_papers.jsonl')